# Tests de funcionalidad de la API de series de tiempo

In [1]:
import os
import pandas as pd
import numpy as np
import requests
from io import StringIO

## Variables

**Obligatorio**: setear la variable de entorno API_URL o setear la variable BASE_URL al ambiente de la API que se quiere probar

In [2]:
BASE_URL = "https://apis-stg.datos.gob.ar/"
METADATA_URL = 'https://apis-stg.datos.gob.ar/series/api/dump/series-tiempo-metadatos.csv'
ENDPOINT_URL = BASE_URL + 'series/api/series/'

In [3]:
series_metadata = pd.read_csv(METADATA_URL)
series_metadata.tail(5)

,catalogo_id,dataset_id,distribucion_id,serie_id,indice_tiempo_frecuencia,serie_titulo,serie_unidades,serie_descripcion,distribucion_titulo,distribucion_descripcion,...,dataset_descripcion,dataset_tema,serie_indice_inicio,serie_indice_final,serie_valores_cant,serie_dias_no_cubiertos,serie_actualizada,serie_valor_ultimo,serie_valor_anterior,serie_var_pct_anterior
19963,sspm,99,99.3,99.3_ING_2008_0_17,R/P1M,ipc_nivel_general,Índice abr-2008=100,Índice de precios al consumidor nivel general....,"Índice de Precios al Consumidor, por grupos. D...",Subsecretaría de Programación Macroeconómica.,...,Índice de Precios al Consumidor del Gran Bueno...,Precios,2006-12-01,2013-12-01,85.0,1798.0,False,166.840000,164.510000,0.014163
19964,sspm,99,99.3,99.3_IR_2008_0_13,R/P1M,ipc_regulados,Índice abr-2008=100,Índice de precios al consumidor regulados. Val...,"Índice de Precios al Consumidor, por grupos. D...",Subsecretaría de Programación Macroeconómica.,...,Índice de Precios al Consumidor del Gran Bueno...,Precios,2006-12-01,2013-12-01,85.0,1798.0,False,161.040000,159.290000,0.010986
19965,sspm,99,99.3,99.3_IR_2008_0_9,R/P1M,ipc_resto,Índice abr-2008=100,Índice de precios al consumidor IPC resto. Val...,"Índice de Precios al Consumidor, por grupos. D...",Subsecretaría de Programación Macroeconómica.,...,Índice de Precios al Consumidor del Gran Bueno...,Precios,2006-12-01,2013-12-01,85.0,1798.0,False,161.740000,159.250000,0.015636
19966,sspm,99,99.3,99.3_PREIR_2008_0_40,R/P1M,precios_relativos_estacionales_ipc_resto,Índice abr-2008=100,Índice de precios al consumidor precios relati...,"Índice de Precios al Consumidor, por grupos. D...",Subsecretaría de Programación Macroeconómica.,...,Índice de Precios al Consumidor del Gran Bueno...,Precios,2006-12-01,2013-12-01,85.0,1798.0,False,137.591196,138.411303,-0.005925
19967,sspm,99,99.3,99.3_PRRIR_2008_0_37,R/P1M,precios_relativos_regulados_ipc_resto,Índice abr-2008=100,Índice de precios al consumidor precios relati...,"Índice de Precios al Consumidor, por grupos. D...",Subsecretaría de Programación Macroeconómica.,...,Índice de Precios al Consumidor del Gran Bueno...,Precios,2006-12-01,2013-12-01,85.0,1798.0,False,99.567207,100.025118,-0.004578


In [4]:
series_metadata.serie_id.count()

19968

## Chequeo de todas las series 

Le pegamos al endpoint para todas las series y verificamos que la API devuelve una respuesta satisfactoria (status code 200). Contamos la cantidad de casos satisfactorios (True) y no (False)

In [5]:
def api_series_head(serie_id):
    return requests.head(ENDPOINT_URL, params={'ids': serie_id}).status_code == 200

### Cantidad de respuestas válidas

In [6]:
valid_responses = series_metadata.serie_id[:10].apply(api_series_head)
valid_responses.value_counts()

True     9
False    1
Name: serie_id, dtype: int64

### En porcentajes

In [7]:
valid_responses.value_counts().apply(lambda x: x/len(valid_responses))

True     0.9
False    0.1
Name: serie_id, dtype: float64

## Chequeo de modos de representación

In [29]:
def api_call(serie, limit=1000, **kwargs):
    call_params = {'ids': serie, 'format': 'csv', 'limit': limit}
    call_params.update(kwargs)
    res = requests.get(ENDPOINT_URL, params=call_params)
    csv = StringIO(res.content.decode('utf8'))
    api_csv = pd.read_csv(csv, parse_dates=['indice_tiempo'], index_col='indice_tiempo')

    return api_csv, res.status_code

def get_source_csv(serie):
    #response = requests.get(ENDPOINT_URL, params={'ids': serie, 'metadata': 'only'}).json()
    #distribution_url = response['meta'][1]['distribution']['downloadURL']
    distribution_url = series_metadata[series_metadata.serie_id == serie]["distribucion_url_descarga"].iloc[0]
    
    #title = response['meta'][1]['field']['title']
    title = series_metadata[series_metadata.serie_id == serie]["serie_titulo"].iloc[0]

    orig_csv = pd.read_csv(distribution_url, parse_dates=['indice_tiempo'], index_col='indice_tiempo')

    return orig_csv, title

### Serie original

In [30]:
# toma un sample de una serie
serie = series_metadata.serie_id.sample(1)

serie_idx = serie.index[0]
serie = serie.values[0]
serie

'8.2_SEGA_2004_T_32'

In [31]:
orig_csv, title = get_source_csv(serie)

### Serie de la API: valor original

In [32]:
api_csv, status_code = api_call(serie)

print("Status code:", status_code)
api_csv.head(1)

Status code: 200


,suministro_electricidad_gas_agua
indice_tiempo,
2004-01-01,7595.81446


In [33]:
orig_csv = orig_csv[:len(api_csv)]
equality_check = np.isclose(orig_csv[title], api_csv[title], equal_nan=True)
equal_df = pd.Series(equality_check)

equal_df.value_counts()

True    59
dtype: int64

### Chequeo de los datos de todas las series

Chequea que los datos que devuelve la API para cada una de las series sean correctos respecto del CSV original del que se toman.

In [34]:
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

In [101]:
def chequear_serie(serie_id):
    """Test completo de chequear una serie contra su CSV original."""
    
    try:
        orig_csv, title = get_source_csv(serie_id)
    except Exception as e:
        return {}, repr(e)
    
    api_csv, status_code = api_call(serie_id)
    
    if isinstance(api_csv, pd.DataFrame):
        orig_csv = orig_csv[:len(api_csv)]
        equality_check = np.isclose(orig_csv[title], api_csv[title], 
                                    rtol=0.001, equal_nan=True)
        equal_df = pd.Series(equality_check)

        return equal_df.value_counts(), status_code

    else:
        return {}, status_code

In [102]:
chequear_serie("89.1_IR_BCRA2_M_0_M_26")

(False    94
 True     93
 dtype: int64, 200)

In [45]:
def chequear_series(series):
    """Testea una lista de series concurrentemente, permitiendo frenar 
    la ejecución con el teclado y obtener resultados parciales."""
    
    results = []

    def chequear_serie_bulk(serie_id):
        res, status_code = chequear_serie(serie_id)
        results.append({
            "serie_id": serie_id, "ok": res.get(True, 0), 
            "error": res.get(False, 0), "status_code": status_code
        })
    
    try:
        with ThreadPoolExecutor(max_workers=7) as executor:
            futures = []
            for serie_id in series:
                fs = executor.submit(chequear_serie_bulk, serie_id)
                futures.append(fs)
            concurrent.futures.wait(futures)
            
    except KeyboardInterrupt:
        return results
    
    return results

In [50]:
series = series_metadata[series_metadata.catalogo_id.isin(["sspm", "energia", "siep"])].serie_id
results = chequear_series(series)
df_results = pd.DataFrame(results)

In [51]:
len(series)

19806

In [52]:
len(df_results)

19795

In [58]:
df_results = df_results.fillna(0)
df_results["error_pct"] = df_results.error / (df_results.error + df_results.ok)

In [106]:
df_results.to_csv("errores-series-valores.csv", encoding="utf8", index=False)

In [105]:
df_results[
    (df_results.error_pct > 0.0001) & 
    (df_results.error > 1.0)
].sort_values("error_pct", ascending=False)

,error,ok,serie_id,status_code,error_pct
343,20.0,0.0,10.1_VIPAA_1993_A_31,200,1.000000
10953,18.0,0.0,367.2_EPEC_DISTRDOR__17,200,1.000000
10947,18.0,0.0,367.2_EMP_DIST_EICA__24,200,1.000000
10948,18.0,0.0,367.2_EMP_ELECTR_SA__24,200,1.000000
10949,18.0,0.0,367.2_ENERGIA_DES.A__24,200,1.000000
10950,18.0,0.0,367.2_ENERGIA_DE_SA__21,200,1.000000
10951,18.0,0.0,367.2_ENERGIA_DE_SA__24,200,1.000000
10952,18.0,0.0,367.2_ENERGIA_SASSA__28,200,1.000000
10954,18.0,0.0,367.2_EPEN_DISTRDOR__17,200,1.000000
10945,18.0,0.0,367.2_EMP_DE_ENE_SA__30,200,1.000000


### Cambio absoluto

In [16]:
api_csv = api_call(serie, representation_mode='change')

In [17]:
orig_csv_change, title = get_source_csv(serie)
orig_csv_change[title] = orig_csv_change[title].diff(1)
orig_csv_change = orig_csv_change[:len(api_csv)]

In [18]:
equality_check = np.isclose(orig_csv_change[title], api_csv[title], equal_nan=True)
equal_df = pd.Series(equality_check)

equal_df.value_counts()

True    56
dtype: int64

### Cambio porcentual

In [19]:
api_csv = api_call(serie, representation_mode='percent_change')

In [20]:
orig_csv_pct_change, title = get_source_csv(serie)
orig_csv_pct_change[title] = orig_csv_pct_change[title].pct_change()
orig_csv_pct_change = orig_csv_pct_change[:len(api_csv)]

In [21]:
equality_check = np.isclose(orig_csv_pct_change[title], api_csv[title], equal_nan=True)
equal_df = pd.Series(equality_check)

equal_df.value_counts()

True    56
dtype: int64

### Colapsos de datos

Aplico las agregaciones máximo y mínimos de la API, y también al CSV original con pandas. Comparo los resultados

In [22]:
api_max = api_call(serie, collapse='year', collapse_aggregation='max')
api_max['api_max'] = api_max[title]
del api_max[title]
api_call(serie).resample('AS').apply(max).join(api_max)

,gtos_cap_transf_cap_ot_1993_2006,api_max
indice_tiempo,,
1993-01-01,16.0,16.0
1994-01-01,14.0,14.0
1995-01-01,33.7,33.7
1996-01-01,7.7,7.7
1997-01-01,21.4,21.4
1998-01-01,19.1,19.1
1999-01-01,16.5,16.5
2000-01-01,14.6,14.6
2001-01-01,9.6,9.6


In [23]:
api_max = api_call(serie, collapse='year', collapse_aggregation='min')
api_max['api_min'] = api_max[title]
del api_max[title]
api_call(serie).resample('AS').apply(min).join(api_max)

,gtos_cap_transf_cap_ot_1993_2006,api_min
indice_tiempo,,
1993-01-01,5.1,5.1
1994-01-01,8.7,8.7
1995-01-01,2.8,2.8
1996-01-01,3.5,3.5
1997-01-01,11.6,11.6
1998-01-01,11.0,11.0
1999-01-01,3.8,3.8
2000-01-01,1.2,1.2
2001-01-01,5.9,5.9


### Metadata

In [24]:
orig_meta = series_metadata.iloc[serie_idx]
orig_meta

catalogo_id                                                               sspm
dataset_id                                                                 375
distribucion_id                                                          375.4
serie_id                                            375.4_GTOS_CAP_T006__32_51
indice_tiempo_frecuencia                                                 R/P3M
serie_titulo                                  gtos_cap_transf_cap_ot_1993_2006
serie_unidades                                               Millones de pesos
serie_descripcion            Gastos de capital transferencias de capital ot...
distribucion_titulo          Organismos Descentralizados. Valores trimestra...
distribucion_descripcion     Esquema Ahorro - Inversión - Financiamiento. O...
distribucion_url_descarga    http://infra.datos.gob.ar/catalog/sspm/dataset...
dataset_responsable              Subsecretaría de Programación Macroeconómica.
dataset_fuente                                      

In [25]:
metadata = requests.get(ENDPOINT_URL, params={'ids': serie, 'metadata': 'full'}).json()

In [26]:
# Metadatos del índice de tiempo
metadata['meta'][0]

{'frequency': 'quarter', 'start_date': '1993-01-01', 'end_date': '2006-10-01'}

In [27]:
serie_meta = metadata['meta'][1]
serie_meta

{'catalog': {'publisher': {'mbox': 'datoseconomicos@mecon.gov.ar',
   'name': 'Subsecretaría de Programación Macroeconómica.'},
  'license': 'Creative Commons Attribution 4.0',
  'description': 'Catálogo de datos abiertos de la Subsecretaría de Programación Macroeconómica.',
  'language': ['SPA'],
  'superThemeTaxonomy': 'http://datos.gob.ar/superThemeTaxonomy.json',
  'issued': '2017-09-28',
  'rights': '2017-09-28',
  'modified': '2017-09-28',
  'spatial': 'ARG',
  'title': 'Datos Programación Macroeconómica',
  'identifier': 'sspm'},
 'dataset': {'publisher': {'mbox': 'datoseconomicos@mecon.gov.ar',
   'name': 'Subsecretaría de Programación Macroeconómica.'},
  'landingPage': 'http://www.minhacienda.gob.ar/secretarias/politica-economica/programacion-macroeconomica/',
  'keyword': ['Información Económica al Día', 'Finanzas Públicas'],
  'superTheme': ['ECON'],
  'title': 'Esquema Ahorro - Inversión - Financimmiento. Organismos Descentralizados. Base Caja.',
  'language': ['SPA'],
  '

Comprobamos que los metadatos de la API sean iguales a los originales

In [36]:
def compare_metadata(serie_meta, original_meta, equivalent_dict_keys):
    comparisons = []
    for key, value in equivalent_dict_keys.items():
        if isinstance(value, dict):
            for api_key, original_key in value.items():
                print(key, api_key, original_key)
                comparison = str(serie_meta[key][api_key]) == str(original_meta[original_key])
                comparisons.append(
                    '{} {} == {}: {} ({} {})'.format(key, api_key, original_key, comparison, serie_meta[key][api_key], original_meta[original_key]))
    return comparisons
    

In [39]:
keys = {
    'catalog': {
        'identifier': 'catalogo_id',
    },
    'dataset': {
        'identifier': 'dataset_id',
    },
    'distribution': {
        'identifier': 'distribucion_id',
        'accrualPeriodicity': 'indice_tiempo_frecuencia',
    },
    'field': {
        'id': 'serie_id',
    },
}

compare_metadata(serie_meta, orig_meta, keys)

catalog identifier catalogo_id
dataset identifier dataset_id
dataset accrualPeriodicity indice_tiempo_frecuencia
distribution identifier distribucion_id
field id serie_id


['catalog identifier == catalogo_id: True (sspm sspm)',
 'dataset identifier == dataset_id: True (375 375)',
 'dataset accrualPeriodicity == indice_tiempo_frecuencia: False (R/P1M R/P3M)',
 'distribution identifier == distribucion_id: True (375.4 375.4)',
 'field id == serie_id: True (375.4_GTOS_CAP_T006__32_51 375.4_GTOS_CAP_T006__32_51)']